# Gold standard curation: Preprocessing and single-step regression

In this stage of gold standard curation, we will do the data preprocessing, selection, and single-step regression for the 153 traits in our question set. The workflow consists of the following steps:

1. Preprocess all the cohorts related to this trait. Each cohort should be converted to a tabular form and saved to a csv file, with columns being genetic factors, the trait, and age, gender if available;
2. If there exists at least one cohort with age or gender information, conduct regression analysis with genetic features together with age or gender as the regressors.


# 1. Basic setup

In [7]:
import os
import sys

sys.path.append('..')
# Set your preferred name
USER = "Yuxuan"
# Set the data and output directories
DATA_ROOT = '/home/yxcheng'
OUTPUT_ROOT = '../../output1'
TRAIT = 'Hypertrophic Cardiomyopathy'

OUTPUT_DIR = os.path.join(OUTPUT_ROOT, USER, '-'.join(TRAIT.split()))
JSON_PATH = os.path.join(OUTPUT_DIR, "cohort_info.json")
if not os.path.exists(OUTPUT_DIR):
    os.makedirs(OUTPUT_DIR, exist_ok=True)

# Gene symbol normalization may take 1-2 minutes. You may set it to False for debugging.
NORMALIZE_GENE = True

In [2]:
# This cell is only for use on Google Colab. Skip it if you run your code in other environments

"""import os
from google.colab import drive

drive.mount('/content/drive', force_remount=True)
proj_dir = '/content/drive/MyDrive/AI4Science_Public'
os.chdir(proj_dir)"""

"import os\nfrom google.colab import drive\n\ndrive.mount('/content/drive', force_remount=True)\nproj_dir = '/content/drive/MyDrive/AI4Science_Public'\nos.chdir(proj_dir)"

# 2. Data preprocessing and selection

## 2.1. The TCGA Xena dataset

In TCGA Xena, there is either zero or one cohort related to the trait. We search the names of subdirectories to see if any matches the trait. If a match is found, we directly obtain the file paths.

In [8]:
import os
from utils import *

dataset = 'TCGA'
dataset_dir = os.path.join(DATA_ROOT, dataset)
os.listdir(dataset_dir)[:10]

['TCGA_Adrenocortical_Cancer_(ACC)',
 'TCGA_Bile_Duct_Cancer_(CHOL)',
 'TCGA_Lower_Grade_Glioma_(LGG)',
 'TCGA_Acute_Myeloid_Leukemia_(LAML)',
 'TCGA_Esophageal_Cancer_(ESCA)',
 'TCGA_Thyroid_Cancer_(THCA)',
 'TCGA_Ocular_melanomas_(UVM)',
 'TCGA_Ovarian_Cancer_(OV)',
 'TCGA_Prostate_Cancer_(PRAD)',
 'CrawlData.ipynb']

If no match is found, jump directly to GEO in Part 2.2

In [7]:
trait_subdir = "TCGA_Breast_Cancer_(BRCA)"
cohort = 'Xena'
# All the cancer traits in Xena are binary
trait_type = 'binary'
# Once a relevant cohort is found in Xena, we can generally assume the gene and clinical data are available
is_available = True

clinical_data_file = os.path.join(dataset_dir, trait_subdir, 'TCGA.BRCA.sampleMap_BRCA_clinicalMatrix')
genetic_data_file = os.path.join(dataset_dir, trait_subdir, 'TCGA.BRCA.sampleMap_HiSeqV2_PANCAN.gz')

In [8]:
import pandas as pd

clinical_data = pd.read_csv(clinical_data_file, sep='\t', index_col=0)
genetic_data = pd.read_csv(genetic_data_file, compression='gzip', sep='\t', index_col=0)
age_col = gender_col = None

In [9]:
_, clinical_data_cols = check_rows_and_columns(clinical_data)

In [10]:
clinical_data_cols[:10]

['AJCC_Stage_nature2012',
 'Age_at_Initial_Pathologic_Diagnosis_nature2012',
 'CN_Clusters_nature2012',
 'Converted_Stage_nature2012',
 'Days_to_Date_of_Last_Contact_nature2012',
 'Days_to_date_of_Death_nature2012',
 'ER_Status_nature2012',
 'Gender_nature2012',
 'HER2_Final_Status_nature2012',
 'Integrated_Clusters_no_exp__nature2012']

Read all the column names in the clinical dataset, to find the columns that record information about age or gender.
Reference prompt:

In [11]:
f'''
Below is a list of column names from a biomedical dataset. Please examine it and identify the columns that are likely to contain information about patients' age. Additionally, please do the same for columns that may hold data on patients' gender. Please provide your answer by strictly following this format, without redundant words:
candidate_age_cols = [col_name1, col_name2, ...]
candidate_gender_cols = [col_name1, col_name2, ...]
If no columns match a criterion, please provide an empty list.

Column names:
{clinical_data_cols}
'''

"\nBelow is a list of column names from a biomedical dataset. Please examine it and identify the columns that are likely to contain information about patients' age. Additionally, please do the same for columns that may hold data on patients' gender. Please provide your answer by strictly following this format, without redundant words:\ncandidate_age_cols = [col_name1, col_name2, ...]\ncandidate_gender_cols = [col_name1, col_name2, ...]\nIf no columns match a criterion, please provide an empty list.\n\nColumn names:\n['AJCC_Stage_nature2012', 'Age_at_Initial_Pathologic_Diagnosis_nature2012', 'CN_Clusters_nature2012', 'Converted_Stage_nature2012', 'Days_to_Date_of_Last_Contact_nature2012', 'Days_to_date_of_Death_nature2012', 'ER_Status_nature2012', 'Gender_nature2012', 'HER2_Final_Status_nature2012', 'Integrated_Clusters_no_exp__nature2012', 'Integrated_Clusters_unsup_exp__nature2012', 'Integrated_Clusters_with_PAM50__nature2012', 'Metastasis_Coded_nature2012', 'Metastasis_nature2012', '

In [12]:
candidate_age_cols = ['Age_at_Initial_Pathologic_Diagnosis_nature2012', 'age_at_initial_pathologic_diagnosis',
                      'days_to_birth', 'year_of_initial_pathologic_diagnosis']
candidate_gender_cols = ['Gender_nature2012', 'gender']

Choose a single column from the candidate columns that record age and gender information respectively.
If no column meets the requirement, keep 'age_col' or 'gender_col' to None

In [13]:
preview_df(clinical_data[candidate_age_cols])

{'Age_at_Initial_Pathologic_Diagnosis_nature2012': [nan, nan, nan, nan, nan],
 'age_at_initial_pathologic_diagnosis': [55.0, 50.0, 62.0, 52.0, 50.0],
 'days_to_birth': [-20211.0, -18538.0, -22848.0, -19074.0, -18371.0],
 'year_of_initial_pathologic_diagnosis': [2004.0,
  2003.0,
  2011.0,
  2011.0,
  2013.0]}

In [14]:
age_col = 'age_at_initial_pathologic_diagnosis'

In [15]:
preview_df(clinical_data[candidate_gender_cols])

{'Gender_nature2012': [nan, nan, nan, nan, nan],
 'gender': ['FEMALE', 'FEMALE', 'FEMALE', 'FEMALE', 'FEMALE']}

In [16]:
gender_col = 'gender'

In [17]:
selected_clinical_data = xena_select_clinical_features(clinical_data, TRAIT, age_col, gender_col)

In [18]:
if NORMALIZE_GENE:
    genetic_data = normalize_gene_symbols_in_index(genetic_data)

In [19]:
merged_data = selected_clinical_data.join(genetic_data.T).dropna()
merged_data.head()

,Epilepsy,Age,Gender,ARHGEF10L,HIF3A,RNF17,RNF10,RNF11,RNF13,GTF2IP1,...,TULP2,NPY5R,GNGT2,GNGT1,TULP3,PTRF,BCL6B,GSTK1,SELP,SELS
sampleID,,,,,,,,,,,,,,,,,,,,,
TCGA-3C-AAAU-01,1,55.0,0.0,0.607308,-3.194126,-0.531035,-0.145872,0.237422,-0.29921,-0.142694,...,-0.748878,0.565583,-0.767233,-1.28139,-0.271377,-0.492286,0.360373,1.067905,0.076267,-0.392212
TCGA-3C-AALI-01,1,50.0,0.0,-0.641192,-4.928226,0.095465,0.098128,-0.541978,-0.32291,-0.044694,...,0.312922,0.079683,0.475267,1.99661,0.161423,0.273714,0.644673,-0.428695,0.068667,-0.043812
TCGA-3C-AALJ-01,1,62.0,0.0,1.082808,-4.623726,-0.531035,0.484028,-0.183678,-0.91901,0.261106,...,-0.748878,-0.656117,-0.216733,0.21081,0.101023,0.660514,1.295073,0.915105,0.168567,0.047788
TCGA-3C-AALK-01,1,52.0,0.0,0.121608,-2.881526,-0.531035,0.179128,0.039222,-0.45491,0.180306,...,0.415422,-0.178417,-0.211233,-1.28139,0.108023,1.132814,0.496773,0.240105,3.099767,0.112888
TCGA-4H-AAAK-01,1,50.0,0.0,0.420208,-3.282726,-0.531035,-0.020972,-0.117978,-0.55781,-0.173794,...,0.685222,-0.698717,-0.484233,-1.28139,-0.223577,1.222714,0.225573,-0.301995,0.200067,-0.080212


In [20]:
print(f"The merged dataset contains {len(merged_data)} samples.")

The merged dataset contains 1215 samples.


In [21]:
is_trait_biased, merge_data = judge_and_remove_biased_features(merged_data, TRAIT, trait_type=trait_type)
is_trait_biased

For the feature 'Epilepsy', the least common label is '0' with 113 occurrences. This represents 9.30% of the dataset.
The distribution of the feature 'Epilepsy' in this dataset is fine.

Quartiles for 'Age':
  25%: 48.0
  50% (Median): 58.0
  75%: 67.0
Min: 26.0
Max: 90.0
The distribution of the feature 'Age' in this dataset is fine.

For the feature 'Gender', the least common label is '1.0' with 13 occurrences. This represents 1.07% of the dataset.
The distribution of the feature 'Gender' in this dataset is fine.



False

In [22]:
merged_data.head()
if not is_trait_biased:
    merge_data.to_csv(os.path.join(OUTPUT_DIR, cohort+'.csv'), index=False)

In [23]:
save_cohort_info(cohort, JSON_PATH, is_available, is_trait_biased, merged_data)

A new JSON file was created at: ../../output1/Haoyang/Epilepsy/cohort_info.json


## 2.2. The GEO dataset

In GEO, there may be one or multiple cohorts for a trait. Each cohort is identified by an accession number. We iterate over all accession numbers in the corresponding subdirectory, preprocess the cohort data, and save them to csv files.

In [9]:
dataset = 'GEO'
trait_subdir = "Hypertrophic-Cardiomyopathy"

trait_path = os.path.join(DATA_ROOT, dataset, trait_subdir)
os.listdir(trait_path)

['GSE2656', 'GSE36961', 'GSE9800', 'GSE36946', 'GSE1145']

Repeat the below steps for all the accession numbers

In [36]:
cohort = accession_num = "GSE36961"
cohort_dir = os.path.join(trait_path, accession_num)
soft_file, matrix_file = get_relevant_filepaths(cohort_dir)
soft_file, matrix_file

('/home/yxcheng/GEO/Hypertrophic-Cardiomyopathy/GSE36961/GSE36961_family.soft.gz',
 '/home/yxcheng/GEO/Hypertrophic-Cardiomyopathy/GSE36961/GSE36961_series_matrix.txt.gz')

### Inital filtering and clinical data preprocessing

In [37]:
background_prefixes = ['!Series_title', '!Series_summary', '!Series_overall_design']
clinical_prefixes = ['!Sample_geo_accession', '!Sample_characteristics_ch1']

background_info, clinical_data = get_background_and_clinical_data(matrix_file, background_prefixes, clinical_prefixes)
print(background_info)

!Series_title	"Transcriptome Profiling of Surgical Myectomy Tissue from Patients with Hypertrophic Cardiomyopathy"
!Series_summary	"Using a high-throughput gene expression profiling technology, we have been able to develop new hypotheses regarding the molecular pathogenic mechanisms of human hypertrophic cardiomyopathy (HCM). It is hoped that these hypotheses, among others generated by this data, will fuel future research endeavors that will uncover novel biomarkers, prognostic indicators, and therapeutic targets to improve our ability to diagnose, counsel, and treat patients with this highly heterogeneous and potentially life-threatening condition."
!Series_overall_design	"Case-control study comparing the messenger RNA transcriptome of cardiac tissues from patients with hypertrophic cardiomyopathy to the transcriptome of control donor cardiac tissues."


In [38]:
clinical_data.head()

,!Sample_geo_accession,GSM907203,GSM907204,GSM907205,GSM907206,GSM907207,GSM907208,GSM907209,GSM907210,GSM907211,...,GSM907338,GSM907339,GSM907340,GSM907341,GSM907342,GSM907343,GSM907344,GSM907345,GSM907346,GSM907347
0,!Sample_characteristics_ch1,Sex: male,Sex: male,Sex: female,Sex: male,Sex: female,Sex: male,Sex: male,Sex: female,Sex: male,...,Sex: female,Sex: female,Sex: female,Sex: male,Sex: female,Sex: female,Sex: male,Sex: male,Sex: male,Sex: male
1,!Sample_characteristics_ch1,age (yrs): 9,age (yrs): 10,age (yrs): 10,age (yrs): 11,age (yrs): 13,age (yrs): 14,age (yrs): 15,age (yrs): 15,age (yrs): 15,...,age (yrs): 53,age (yrs): 48,age (yrs): 47,age (yrs): 4,age (yrs): 48,age (yrs): 25,age (yrs): 27,age (yrs): 21,age (yrs): 27,age (yrs): 21
2,!Sample_characteristics_ch1,tissue: cardiac,tissue: cardiac,tissue: cardiac,tissue: cardiac,tissue: cardiac,tissue: cardiac,tissue: cardiac,tissue: cardiac,tissue: cardiac,...,tissue: cardiac,tissue: cardiac,tissue: cardiac,tissue: cardiac,tissue: cardiac,tissue: cardiac,tissue: cardiac,tissue: cardiac,tissue: cardiac,tissue: cardiac
3,!Sample_characteristics_ch1,disease state: hypertrophic cardiomyopathy (HCM),disease state: hypertrophic cardiomyopathy (HCM),disease state: hypertrophic cardiomyopathy (HCM),disease state: hypertrophic cardiomyopathy (HCM),disease state: hypertrophic cardiomyopathy (HCM),disease state: hypertrophic cardiomyopathy (HCM),disease state: hypertrophic cardiomyopathy (HCM),disease state: hypertrophic cardiomyopathy (HCM),disease state: hypertrophic cardiomyopathy (HCM),...,sample type: control,sample type: control,sample type: control,sample type: control,sample type: control,sample type: control,sample type: control,sample type: control,sample type: control,sample type: control
4,!Sample_characteristics_ch1,sample type: case,sample type: case,sample type: case,sample type: case,sample type: case,sample type: case,sample type: case,sample type: case,sample type: case,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [39]:
clinical_data_unique = get_unique_values_by_row(clinical_data)
clinical_data_unique

{0: ['Sex: male', 'Sex: female'],
 1: ['age (yrs): 9',
  'age (yrs): 10',
  'age (yrs): 11',
  'age (yrs): 13',
  'age (yrs): 14',
  'age (yrs): 15',
  'age (yrs): 16',
  'age (yrs): 17',
  'age (yrs): 19',
  'age (yrs): 20',
  'age (yrs): 23',
  'age (yrs): 26',
  'age (yrs): 27',
  'age (yrs): 28',
  'age (yrs): 30',
  'age (yrs): 31',
  'age (yrs): 32',
  'age (yrs): 33',
  'age (yrs): 35',
  'age (yrs): 37',
  'age (yrs): 38',
  'age (yrs): 41',
  'age (yrs): 43',
  'age (yrs): 44',
  'age (yrs): 45',
  'age (yrs): 46',
  'age (yrs): 47',
  'age (yrs): 48',
  'age (yrs): 50',
  'age (yrs): 51'],
 2: ['tissue: cardiac', 'sample type: control'],
 3: ['disease state: hypertrophic cardiomyopathy (HCM)',
  nan,
  'sample type: control'],
 4: ['sample type: case', nan]}

Analyze the metadata to determine data relevance and find ways to extract the clinical data.
Reference prompt:

In [40]:
f'''As a biomedical research team, we are selecting datasets to study the association between the human trait \'{TRAIT}\' and genetic factors, optionally considering the influence of age and gender. After searching the GEO database and parsing the matrix file of a series, we obtained background information and sample characteristics data. We will provide textual information about the dataset background, and a Python dictionary storing a list of unique values for each field of the sample characteristics data. Please carefully review the provided information and answer the following questions about this dataset:
1. Does this dataset contain gene expression data? (Note: Pure miRNA data is not suitable.)
2. For each of the traits \'{TRAIT}\', 'age', and 'gender', please address these points:
   (1) Is there human data available for this trait?
   (2) If so, identify the key in the sample characteristics dictionary where unique values of this trait is recorded. The key is an integer. The trait information might be explicitly recorded, or can be inferred from the field with some biomedical knowledge or understanding about the data collection process.
   (3) Choose an appropriate data type (either 'continuous' or 'binary') for each trait. Write a Python function to convert any given value of the trait to this data type. The function should handle inference about the trait value and convert unknown values to None.
   Name the functions 'convert_trait', 'convert_age', and 'convert_gender', respectively.

Background information about the dataset:
{background_info}

Sample characteristics dictionary (from "!Sample_characteristics_ch1", converted to a Python dictionary that stores the unique values for each field):
{clinical_data_unique}
'''

'As a biomedical research team, we are selecting datasets to study the association between the human trait \'Hypertrophic Cardiomyopathy\' and genetic factors, optionally considering the influence of age and gender. After searching the GEO database and parsing the matrix file of a series, we obtained background information and sample characteristics data. We will provide textual information about the dataset background, and a Python dictionary storing a list of unique values for each field of the sample characteristics data. Please carefully review the provided information and answer the following questions about this dataset:\n1. Does this dataset contain gene expression data? (Note: Pure miRNA data is not suitable.)\n2. For each of the traits \'Hypertrophic Cardiomyopathy\', \'age\', and \'gender\', please address these points:\n   (1) Is there human data available for this trait?\n   (2) If so, identify the key in the sample characteristics dictionary where unique values of this tra

Understand and verify the answer from GPT, to assign values to the below variables. Assign None to the 'row_id' variables if relevant data row was not found.
Later we need to let GPT format its answer to automatically do these. But given the complexity of this step, let's grow some insight from the free-text answers for now.

In [41]:
is_gene_availabe = True
trait_row = 3
age_row = 1
gender_row = 0

trait_type = 'binary'

In [42]:
is_available = is_gene_availabe and (trait_row is not None)
if not is_available:
    save_cohort_info(cohort, JSON_PATH, is_available)
    print("This cohort is not usable. Please skip the following steps and jump to the next accession number.")

In [43]:
def convert_trait(tissue_type):
    """
    Convert tissue type to epilepsy presence (binary).
    Assuming epilepsy presence for 'Hippocampus' tissue.
    """
    if tissue_type == 'disease state: hypertrophic cardiomyopathy (HCM)':
        return 1  # Epilepsy present
    else:
        return 0  # Epilepsy not present

def convert_age(age_string):
    """
    Convert age string to a numerical value.
    """
    if age_string.startswith('age (yrs): '):
        try:
            return int(age_string.split('age (yrs): ')[1].rstrip(''))  # Extracting numeric part
        except ValueError:
            return None  # In case of parsing error
    else:
        return None  # If age string does not follow expected format

def convert_gender(gender_string):
    """
    Convert gender string to a binary representation.
    Female -> 0, Male -> 1, Unknown -> None
    """
    if gender_string == 'Sex: male':
        return 0
    elif gender_string == 'Sex: female':
        return 1
    else:
        return None  # In case of unknown gender

# Example usage
example_tissue = 'tissue: Hippocampus'
example_age = 'age: 23y'
example_gender = 'gender: Male'

converted_trait = convert_trait(example_tissue)
converted_age = convert_age(example_age)
converted_gender = convert_gender(example_gender)

converted_trait, converted_age, converted_gender

(0, None, None)

In [44]:
selected_clinical_data = geo_select_clinical_features(clinical_data, TRAIT, trait_row, convert_trait, age_row=age_row,
                                                      convert_age=convert_age, gender_row=gender_row,
                                                      convert_gender=convert_gender)
selected_clinical_data.head()

/home/yxcheng/AI4Science_gold_standard/stage1/Yuxuan/../utils.py:208: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  clinical_df = clinical_df.applymap(convert_fn)
/home/yxcheng/AI4Science_gold_standard/stage1/Yuxuan/../utils.py:208: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  clinical_df = clinical_df.applymap(convert_fn)
/home/yxcheng/AI4Science_gold_standard/stage1/Yuxuan/../utils.py:208: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  clinical_df = clinical_df.applymap(convert_fn)


,GSM907203,GSM907204,GSM907205,GSM907206,GSM907207,GSM907208,GSM907209,GSM907210,GSM907211,GSM907212,...,GSM907338,GSM907339,GSM907340,GSM907341,GSM907342,GSM907343,GSM907344,GSM907345,GSM907346,GSM907347
Hypertrophic Cardiomyopathy,1,1,1,1,1,1,1,1,1,1,...,0,0,0,0,0,0,0,0,0,0
Age,9,10,10,11,13,14,15,15,15,15,...,53,48,47,4,48,25,27,21,27,21
Gender,0,0,1,0,1,0,0,1,0,0,...,1,1,1,0,1,1,0,0,0,0


### Genetic data preprocessing and final filtering

In [45]:
genetic_data = get_genetic_data(matrix_file)
genetic_data.head()

,GSM907203,GSM907204,GSM907205,GSM907206,GSM907207,GSM907208,GSM907209,GSM907210,GSM907211,GSM907212,...,GSM907338,GSM907339,GSM907340,GSM907341,GSM907342,GSM907343,GSM907344,GSM907345,GSM907346,GSM907347
ID,,,,,,,,,,,,,,,,,,,,,
7A5,6.084081,5.900519,6.051991,6.108818,5.998926,6.325295,5.837986,6.010667,5.847576,5.900796,...,6.026094,5.950691,5.991524,5.884627,5.995006,5.958711,6.056444,5.952569,5.647039,6.121388
A1BG,5.917069,6.198242,6.202327,6.293233,6.207701,6.352847,6.006204,6.251114,6.523222,6.155067,...,6.278075,6.050261,6.158821,6.218785,6.214677,6.123679,6.165580,6.304735,6.207726,6.341202
A1CF,6.050514,6.316973,6.135774,6.053267,6.089185,6.175524,6.069108,6.090359,6.137273,6.005156,...,6.145559,6.222298,5.978435,6.159344,5.928987,6.032816,6.060230,6.120658,5.937447,6.244038
A26A1,6.068293,6.020148,6.215063,6.162022,6.137995,5.963461,6.033446,6.230270,6.185026,6.186375,...,6.084727,6.079091,6.050476,5.873180,6.014934,6.046725,6.072258,6.028381,6.152996,6.042913
A26B1,6.145956,6.126624,5.827730,5.974526,5.843412,5.993611,6.258971,6.191147,5.983448,6.073970,...,6.072644,6.011094,5.683178,6.134940,6.106568,5.905240,5.991379,6.219802,5.863304,6.018874


In [46]:
gene_row_ids = genetic_data.index[:20].tolist()
gene_row_ids

['7A5',
 'A1BG',
 'A1CF',
 'A26A1',
 'A26B1',
 'A26C1B',
 'A26C3',
 'A2BP1',
 'A2M',
 'A2ML1',
 'A3GALT2',
 'A4GALT',
 'A4GNT',
 'AAA1',
 'AAAS',
 'AACS',
 'AACSL',
 'AADAC',
 'AADACL1',
 'AADACL2']

Check if the gene dataset requires mapping to get the gene symbols corresponding to each data row.

Reference prompt:

In [47]:
f'''
Below are the row headers of a gene expression dataset in GEO. Based on your biomedical knowledge, are they human gene symbols, or are they some other identifiers that need to be mapped to gene symbols? Your answer should be concluded by starting a new line and strictly following this format:
requires_gene_mapping = (True or False)

Row headers:
{gene_row_ids}
'''

"\nBelow are the row headers of a gene expression dataset in GEO. Based on your biomedical knowledge, are they human gene symbols, or are they some other identifiers that need to be mapped to gene symbols? Your answer should be concluded by starting a new line and strictly following this format:\nrequires_gene_mapping = (True or False)\n\nRow headers:\n['7A5', 'A1BG', 'A1CF', 'A26A1', 'A26B1', 'A26C1B', 'A26C3', 'A2BP1', 'A2M', 'A2ML1', 'A3GALT2', 'A4GALT', 'A4GNT', 'AAA1', 'AAAS', 'AACS', 'AACSL', 'AADAC', 'AADACL1', 'AADACL2']\n"

If not required, jump directly to the gene normalization step

In [48]:
requires_gene_mapping = True

In [49]:
if requires_gene_mapping:
    gene_annotation = get_gene_annotation(soft_file)
    gene_annotation_summary = preview_df(gene_annotation)
    print(gene_annotation_summary)

{'ID': ['7A5', 'A1BG', 'A1CF', 'A26A1', 'A26B1'], 'ORF': ['7A5', 'A1BG', 'A1CF', 'A26A1', 'A26B1'], 'GENE_ID': [nan, nan, nan, nan, nan], 'UNIGENE_ID': [nan, nan, nan, nan, nan], 'SPOT_ID': [nan, nan, nan, nan, nan]}


Observe the first few cells in the ID column of the gene annotation dataframe, to find the names of columns that store the gene probe IDs and gene symbols respectively.
Reference prompt:

In [50]:
if requires_gene_mapping:
    print(f'''
    As a biomedical research team, we extracted the gene annotation data from a series in the GEO database, and saved it to a Python dictionary. Please read the dictionary, and decide which key stores the ID of the probe, and which key stores the gene symbols. Please strict follow this format in your answer:
    probe_name_key = 'key_name1'
    gene_name_key = 'key_name2'

    Gene annotation dictionary:
    {gene_annotation_summary}
    ''')


    As a biomedical research team, we extracted the gene annotation data from a series in the GEO database, and saved it to a Python dictionary. Please read the dictionary, and decide which key stores the ID of the probe, and which key stores the gene symbols. Please strict follow this format in your answer:
    probe_name_key = 'key_name1'
    gene_name_key = 'key_name2'

    Gene annotation dictionary:
    {'ID': ['7A5', 'A1BG', 'A1CF', 'A26A1', 'A26B1'], 'ORF': ['7A5', 'A1BG', 'A1CF', 'A26A1', 'A26B1'], 'GENE_ID': [nan, nan, nan, nan, nan], 'UNIGENE_ID': [nan, nan, nan, nan, nan], 'SPOT_ID': [nan, nan, nan, nan, nan]}
    


In [51]:
if requires_gene_mapping:
    probe_id_key = 'ID'
    gene_symb_key = 'ORF'
    gene_mapping = get_gene_mapping(gene_annotation, probe_id_key, gene_symb_key)
    genetic_data = apply_gene_mapping(genetic_data, gene_mapping)

In [52]:
if NORMALIZE_GENE:
    genetic_data = normalize_gene_symbols_in_index(genetic_data)

1000 input query terms found no hit:	['10.00000165', '10.00002174', '10.0000285', '10.00003048', '10.00003353', '10.00004374', '10.000051
1000 input query terms found no hit:	['10.00661048', '10.00662491', '10.00664978', '10.00664999', '10.00666058', '10.00666091', '10.00666
1000 input query terms found no hit:	['10.01364396', '10.01364612', '10.01364918', '10.01365441', '10.01366376', '10.01366564', '10.01367
1000 input query terms found no hit:	['10.02058597', '10.02058945', '10.02059316', '10.02059541', '10.02059615', '10.02059962', '10.02060
1000 input query terms found no hit:	['10.02693495', '10.02693894', '10.02694465', '10.02694506', '10.02695028', '10.0269578', '10.026959
1000 input query terms found no hit:	['10.03393869', '10.03394834', '10.03396291', '10.03396626', '10.0339709', '10.03397225', '10.033978
1000 input query terms found no hit:	['10.04073365', '10.04073582', '10.04073699', '10.04074788', '10.04075608', '10.04076108', '10.04077
1000 input query terms found no hi

KeyboardInterrupt: 

In [31]:
merged_data = geo_merge_clinical_genetic_data(selected_clinical_data, genetic_data)
# The preprocessing runs through, which means is_available should be True
is_available = True

In [32]:
print(f"The merged dataset contains {len(merged_data)} samples.")
merged_data

The merged dataset contains 124 samples.


,Hypertrophic Cardiomyopathy,Age,Gender
GSM906828,1.0,9.0,0.0
GSM906829,1.0,10.0,0.0
GSM906830,1.0,10.0,1.0
GSM906831,1.0,11.0,0.0
GSM906832,1.0,13.0,1.0
...,...,...,...
GSM906950,0.0,55.0,0.0
GSM906951,0.0,61.0,0.0
GSM906952,0.0,21.0,0.0
GSM906953,0.0,27.0,0.0


In [33]:
is_trait_biased, merged_data = judge_and_remove_biased_features(merged_data, TRAIT, trait_type=trait_type)
is_trait_biased

For the feature 'Hypertrophic Cardiomyopathy', the least common label is '0.0' with 17 occurrences. This represents 13.71% of the dataset.
The distribution of the feature 'Hypertrophic Cardiomyopathy' in this dataset is fine.

Quartiles for 'Age':
  25%: 31.75
  50% (Median): 48.5
  75%: 59.0
Min: 9.0
Max: 78.0
The distribution of the feature 'Age' in this dataset is fine.

For the feature 'Gender', the least common label is '1.0' with 58 occurrences. This represents 46.77% of the dataset.
The distribution of the feature 'Gender' in this dataset is fine.



False

In [34]:
save_cohort_info(cohort, JSON_PATH, is_available, is_trait_biased, merged_data, note='')

In [35]:
merged_data.head()
if not is_trait_biased:
    merged_data.to_csv(os.path.join(OUTPUT_DIR, cohort+'.csv'), index=False)

### 3. Do regression & Cross Validation

In [4]:
from utils import *

In [5]:
best_cohort, ranked_df = filter_and_rank_cohorts(JSON_PATH, 'has_age')
best_cohort

'GSE36961'

In [6]:
ranked_df.head()

,cohort_id,is_usable,is_available,is_biased,has_age,has_gender,sample_size,note
1,GSE36961,True,True,False,True,True,142,
0,GSE36946,True,True,False,True,True,124,


In [ ]:
ck=10000
listq=[]
for df in pd.read_csv(os.path.join(OUTPUT_DIR, best_cohort+'.csv'),chunksize=ck):
    listq.append(df)
merged_data = pd.concat(listq)
#merged_data = pd.read_csv(os.path.join(OUTPUT_DIR, best_cohort+'.csv'))

In [53]:
merged_data.head()

,Hypertrophic Cardiomyopathy,Age,Gender,10.00003418,10.00015128,10.000219,10.00025391,10.00027026,10.00028059,10.00035325,...,ILMN_3168911,ILMN_3168912,ILMN_3168913,ILMN_3168914,ILMN_3168915,ILMN_3168916,ILMN_3168917,ILMN_3168918,ILMN_3168919,ILMN_3168920
0,1.0,9.0,0.0,9.606022,9.311694,9.873775,10.621399,10.255539,10.282637,9.552235,...,7.893887,8.659658,7.511333,12.986162,10.838765,8.670022,12.997682,9.044866,9.111395,10.524959
1,1.0,10.0,0.0,8.874975,9.078207,10.261361,10.900479,10.070205,10.443853,9.259145,...,7.631946,7.599127,7.501344,13.521643,7.943698,9.008603,13.263010,9.159761,9.477100,10.425323
2,1.0,10.0,1.0,9.291308,9.639385,9.902157,10.269030,10.333924,9.955272,10.104539,...,8.890307,8.339787,7.114740,13.367404,9.719279,8.789594,13.442037,11.134543,9.446751,10.815096
3,1.0,11.0,0.0,8.631738,8.937598,9.950913,9.714647,9.919599,9.251646,9.812546,...,8.255842,8.508035,7.243051,13.147486,10.372047,8.712316,13.375907,10.783961,9.352083,10.408644
4,1.0,13.0,1.0,7.945530,9.610182,10.021578,10.497897,10.259784,9.897997,9.745380,...,7.685630,8.756333,7.149065,13.124336,10.437644,8.906580,13.156568,9.027919,9.353549,10.658034


In [54]:
# If both age and gender features are available, select 'age' as the condition.
condition = 'Age'
# Remove the other condition to prevent interference.
merged_data = merged_data.drop(columns=['Gender'], errors='ignore').astype('float')

In [55]:
X = merged_data.drop(columns=[TRAIT, condition]).values
Y = merged_data[TRAIT].values
Z = merged_data[condition].values

Select the appropriate regression model depending on whether the dataset shows batch effect.

In [56]:
has_batch_effect = detect_batch_effect(X)
has_batch_effect

True

In [57]:
if has_batch_effect:
    model_constructor = VariableSelection
    model_params = {}
else:
    model_constructor = Lasso
    model_params = {'alpha': 1.0, 'random_state': 42}

In [58]:
cv_mean, cv_std = cross_validation(X, Y, Z, model_constructor, model_params, target_type=trait_type)

The cross-validation accuracy is 67.50% ± 7.17%


In [59]:
# Train regression model on the whole dataset to identify significant genes
model = ResidualizationRegressor(model_constructor, model_params)
normalized_X, _ = normalize_data(X)
normalized_Z, _ = normalize_data(Z)
model.fit(normalized_X, Y, normalized_Z)

### 5. Discussion and report

In [61]:
feature_cols = merged_data.columns.tolist()
feature_cols.remove(TRAIT)

if has_batch_effect:
    report_result_from_lmm(model, feature_cols, TRAIT, condition, threshold=0.05, save_output=True,
                           output_dir=OUTPUT_DIR)
else:
    report_result_from_lasso(model, feature_cols, TRAIT, condition, save_output=True, output_dir=OUTPUT_DIR)

Effect of the condition on the target variable:
Variable: Age
Coefficient: 0.0296
Found 0 significant genes associated with the trait 'Hypertrophic Cardiomyopathy' conditional on the factor 'Age', with corrected p-value < 0.05:
Empty DataFrame
Columns: [Variable, Coefficient, corrected_p_value]
Index: []


/Users/hxwxss/AI4Science_gold_standard/stage1/Yuxuan/utils.py:425: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  gene_regression_df.loc[:, 'corrected_p_value'] = corrected_p_values
